In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/newsdataset/preprocessed_data.csv


In [9]:
!pip install sent2vec
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 7.4 MB/s eta 0:00:00


In [48]:
from ast import Num
import torch
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch.nn.functional as F
# from utils import load_data, sentence_embedding, label_encoding

class NewsSentenceLevelClassificationDataset(Dataset):
    def __init__(self,data_path,label_path,num,mode):
        self.data_path = data_path
        self.label_path = label_path
        self.mode = mode
        self.num = num
        #load dataset
        title, classes, class_count = load_pickle(self.data_path,self.label_path,self.num,self.mode)
        #sentence embedding
        #embedded_title = sentence_embedding(title)

        #label encoding

        #label_encode= label_encoding(classes)
        self.data = title
        self.classes = classes
        self.num_class = class_count
        print(len(self.data))

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        X = self.data[idx]
        Y = F.one_hot(torch.tensor(self.classes[idx]), num_classes=self.num_class)
        return X, Y


In [51]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.preprocessing import LabelEncoder
#from sent2vec.vectorizer import Vectorizer
import pickle
import sent2vec
import numpy as np


import sys, os

def load_pickle(data_path,label_path,num,mode):

    data_file = open(data_path,"rb")
    label_file = open(label_path,"rb")
    data =pickle.load(data_file)
    label = pickle.load(label_file)

    class_count =np.unique(label).shape[0]

    length =int(len(data) * num)
    if mode == "train":
       return data[:length],label[:length],class_count
    elif mode== "valid":
      length = int(len(data)*(1-num))
      return data[length:], label[length:], class_count
    else:
        return data, label, class_count
# def load_data(path,num):
#     df =   pd.read_csv(path)
#     df = df.dropna()
#     df = df.sample(n=num)
#     title = list(df['title'])
#     num_classes = len(df['class'].unique())
#     classes =list(df['class'])
#     return title, classes, num_classes

# def sentence_embedding(title):
#     string = []
#     string.append(title)


#     #stop_words = list(set(stopwords))
#     blockPrint()
#     vector_obj =Vectorizer()
#     sentence_vector = []
#     #print("Size of the length",type(title))
#     enablePrint()
#     vector_obj.run(string)

#     vectors = vector_obj.vectors
#     for vec in vectors :
#         sentence_vector.append(vec)
#     return sentence_vector


# def label_encoding(classes):
#     cls = []
#     cls.append(classes)
#     le = LabelEncoder()
#     return le.fit_transform(cls)


In [52]:
#from Dataset import NewsSentenceLevelClassificationDataset
from torch.utils.data import DataLoader
import gc
torch.cuda.empty_cache()
gc.collect()


training_data = NewsSentenceLevelClassificationDataset(data_path="/content/drive/MyDrive/NEWS Classification/new_embedding.pkl",label_path="/content/drive/MyDrive/NEWS Classification/label.pkl",num=0.9,mode="train")
testing_data = NewsSentenceLevelClassificationDataset(data_path="/content/drive/MyDrive/NEWS Classification/new_embedding.pkl",label_path="/content/drive/MyDrive/NEWS Classification/label.pkl",num=0.1,mode="train")
train_dataloader = DataLoader(training_data, batch_size=256, shuffle=True)
test_dataloader = DataLoader(testing_data,batch_size=256,shuffle=True)
#test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)



250365
27818


In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
import torch.nn as nn

class NewsClassificationModel(nn.Module):

    def __init__(self,num_classes):
        super(NewsClassificationModel, self).__init__()
        self.lstm =  nn.LSTM(input_size=768,hidden_size=64, bidirectional=True,batch_first=True)
        self.dense = nn.Linear(64*2,num_classes)
        self.dropout = nn.Dropout(p=0.2)
        self.softmax = nn.Softmax(dim=1)
        self.lossfn = nn.CrossEntropyLoss()
    def forward(self, x):
        x,_ = self.lstm(x)
        x = self.dropout(x)
        x = self.dense(x)
        x = self.softmax(x)

        return x

    def loss(self,data,label):
        prediction =self.forward(data)
#         print(prediction.shape,label.shape)
#         label = label.type(torch.FloatTensor)
        return prediction,self.lossfn(prediction,label)

    def predict(self,data):
        return self.forward(data)


In [ ]:
#from Classifier.Model.SentenceClassification import SentenceClassification
import torch
from  torch.optim import AdamW
#model intialization
from torchmetrics.classification import F1Score
from tqdm import tqdm
f1 = F1Score(task="multiclass", num_classes=27)

def init():
    model = NewsClassificationModel(27)
    return model

def training_step(model,data,label):
    model.train()
    prediction,loss = model.loss(data,label)
    data = data.detach().cpu().numpy()
    label = label.detach().cpu().numpy()
    prediction = prediction.detach().cpu().numpy()
    prediction_map = []
    label_map = []
    for i  in range(len(prediction)):
        prediction_index = np.argmax(prediction[i])
        label_index = np.argmax(label[i])
        prediction_map.append(prediction_index)
        label_map.append(label_index)
    prediction_map = np.array(prediction_map,dtype=float)
    label_map = np.array(label_map,dtype=float)
    prediction_map = torch.from_numpy(prediction_map)
    label_map = torch.from_numpy(label_map)
    f1_score = f1(prediction_map,label_map)

    return loss,f1_score

def validation_step(model,data,label):
    model.eval()
    prediction,loss =model.loss(data,label)
    data = data.detach().cpu().numpy()
    label = label.detach().cpu().numpy()
    prediction = prediction.detach().cpu().numpy()
    prediction_map = []
    label_map = []
    for i  in range(len(prediction)):
        prediction_index = np.argmax(prediction[i])
        label_index = np.argmax(label[i])
        prediction_map.append(prediction_index)
        label_map.append(label_index)
    prediction_map = np.array(prediction_map,dtype=float)
    label_map = np.array(label_map,dtype=float)
    prediction_map = torch.from_numpy(prediction_map)
    label_map = torch.from_numpy(label_map)
#     print(prediction_map,label_map)
    f1_score = f1(prediction_map,label_map)
    return loss, f1_score

#begining
model = init()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(model)

print(device)
optimizer = AdamW(model.parameters(),lr=1e-4)


epochs = 150

for epoch in  range(0,epochs):
    print('Epoch',epoch)
    loss_train = []
    score_train = []

    for data,label in train_dataloader:
        optimizer.zero_grad()

        label = label.type(torch.FloatTensor)
        data = data.to(device)
        label = label.to(device)
        loss,f1_score = training_step(model,data,label)
        loss.backward()
        optimizer.step()


        loss_train.append(loss.item())
        score_train.append(f1_score)

    for data,label in test_dataloader:
        optimizer.zero_grad()
        label = label.type(torch.FloatTensor)
        data = data.to(device)
        label = label.to(device)
        loss,f1_score = training_step(model,data,label)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        loss.backward()
        optimizer.step()



        loss_train.append(loss.item())
        score_train.append(f1_score)

    loss_val =[]
    score_val = []
    for data,label in test_dataloader:

        label = label.type(torch.FloatTensor)
        data = data.to(device)
        label = label.to(device)

        loss,f1_score = validation_step(model,data,label)
        loss_val.append(loss.item())
        score_val.append(f1_score)
    loss_val = sum(loss_val)/len(loss_val)
    loss_train = sum(loss_train)/len(loss_train)
    score_train = sum(score_train)/len(score_train)
    score_val = sum(score_val)/len(score_val)
    torch.save({"model":model.state_dict(),
                            "o_model" : model,
                             "optimizer":optimizer.state_dict(),
                             "f1-score_training":  score_train,
                             "f1-score_val":score_val,
                             "loss_train":loss_val,
                             "loss_val" : loss_train
                           },'/content/drive/MyDrive/NEWS Classification/Models/Model-Layer-Stopping-Epoch:'+str(epoch+1)+'.pt')
                  # print("Early stopping called and stopped the training.")
    print("Epoch",(epoch+1),' Training Loss:', loss_train, ' F1-score',score_train,'Validation Loss:',loss_val,'F1-score',score_val)

NewsClassificationModel(
  (lstm): LSTM(768, 64, batch_first=True, bidirectional=True)
  (dense): Linear(in_features=128, out_features=27, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (softmax): Softmax(dim=1)
  (lossfn): CrossEntropyLoss()
)
cuda
Epoch 0
Epoch 1  Training Loss: 3.064839682890498  F1-score tensor(0.3439) Validation Loss: 2.774061769520471 F1-score tensor(0.6422)
Epoch 1
Epoch 2  Training Loss: 2.891113246814198  F1-score tensor(0.5151) Validation Loss: 2.6503211489511194 F1-score tensor(0.7574)
Epoch 2
Epoch 3  Training Loss: 2.849776311214626  F1-score tensor(0.5317) Validation Loss: 2.6208725281811636 F1-score tensor(0.7653)
Epoch 3
Epoch 4  Training Loss: 2.835982854802782  F1-score tensor(0.5372) Validation Loss: 2.6070793440582554 F1-score tensor(0.7711)
Epoch 4
Epoch 5  Training Loss: 2.8280980297481673  F1-score tensor(0.5409) Validation Loss: 2.598710932862868 F1-score tensor(0.7765)
Epoch 5
Epoch 6  Training Loss: 2.822620891055166  F1-score tensor(

In [ ]:
xx!  pip install torch==1.13.1

In [ ]:
#sentence embedding code
import pickle
import pandas as pd

jump = 256
def sentence_embedding(text_list):
  f = open("embedding.pkl","ab")
  for i in range(0,len(text_list),jump):
    vector_obj =Vectorizer()
    sentence_vector = []
    vector_obj.run(text_list[i:i+jump])

    vectors = vector_obj.vectors
    for vec in vectors :
        sentence_vector.append(vec)
    # sentence_vector
    print("Dumped",i, i+jump)
    pickle.dump(sentence_vector,f)


df = pd.read_csv("/content/drive/MyDrive/NEWS Classification/preprocessed_data.csv")
df = df.dropna()
# df = df.sample(n=num)
title = list(df['title'])
sentence_embedding(title)

KeyboardInterrupt: ignored

In [32]:
from sklearn import preprocessing
import pickle
import pandas as pd
# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
df = pd.read_csv("/content/drive/MyDrive/NEWS Classification/preprocessed_data.csv")
df = df.dropna()
classes= label_encoder.fit_transform(df["class"])
classes = np.array(classes)
f = open("/content/drive/MyDrive/NEWS Classification/label.pkl","wb")
pickle.dump(classes,f)

In [34]:
f = open("/content/drive/MyDrive/NEWS Classification/new_embedding.pkl","rb")
fb = open("/content/drive/MyDrive/NEWS Classification/label.pkl","rb")
result =pickle.load(f)
result1 = pickle.load(fb)

print(len(result),result1.shape)

278184 (278184,)
